In [ ]:
import bmll2 as b2
from bmll2 import reference, Security, NormalisedSecurity, SparkHelper, get_market_data, get_market_data_range, VenueMarketError, get_market_tables, save_spark_dataframe, load_spark_dataframe
b2.get_file('modules/auxiliary_functions.py')

In [ ]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype
import gc

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy.stats import t

## Data processing for plotting the convex post execution decay

In [ ]:
ticker = 'GRT'

In [ ]:
b2.get_file('top_100(Volume)/GRT.csv')
stock_data = pd.read_csv('GRT.csv', parse_dates = ['DateTime', 'Date'])
stock_data = stock_data.rename(columns = {'Ticker' : 'RIC'})
stock_data = stock_data.sort_values(['DateTime', 'ExchangeSequenceNo'])


In [ ]:
b2.get_file('test_data/homo_4_immediate(100).csv')
impact_data = pd.read_csv('homo_4_immediate(100).csv', parse_dates = ['Date', 'Start time', 'End time'])

stock_AD_data = impact_data[impact_data['RIC'] == ticker][['RIC', 'Date', '20 AD volatility', '20 AD volume']]
stock_AD_data = stock_AD_data.drop_duplicates(subset = ['Date']).reset_index(drop = True)

del impact_data
gc.collect()


In [ ]:
def impact_df_decay(metaorders_list, timing_method = 'immediate'): 
    
    num_trades = len(metaorders_list)
    
    features = pd.DataFrame(columns = ['RIC', 'Date', 'Start time', 'End time', 'daily volume', 'intraday volatility',
                                       'number child orders', 'volume traded', 'trade sign', 'impact(simple)', 'Mid-price before', 'Mid-price after(immediate)'])
    for i in range(num_trades):
        metaorder = metaorders_list[i]

        if metaorder.empty:
            continue

        intention  = metaorder.iloc[0]['Trade Sign']
        ave_impact = af.impact(metaorder, timing_method = timing_method, impact_method = 'simple')
        n          = metaorder.shape[0]
        volume     = sum(metaorder.loc[:, 'Volume'])

        features.at[i, 'RIC']                       = metaorder['RIC'].iloc[0]
        features.at[i, 'Date']                      = metaorder['Date'].iloc[0]
        features.at[i, 'Start time']                = metaorder['DateTime'].iloc[0]
        features.at[i, 'End time']                  = metaorder['DateTime'].iloc[-1]
        features.at[i, 'daily volume']              = metaorder['Daily Volume'].iloc[0]
        features.at[i, 'intraday volatility']       = metaorder['Daily Volatility'].iloc[0]
        features.at[i, 'number child orders']       = n
        features.at[i, 'volume traded']             = volume
        features.at[i, 'trade sign']                = intention
        features.at[i, 'impact(simple)']            = ave_impact
        features.at[i, 'Mid-price before']          = metaorder['Mid-price before'].iloc[0]
        features.at[i, 'Mid-price after(immediate)'] = metaorder['Mid-price after(immediate)'].iloc[-1]
        

    return features


In [ ]:
%%time
stock_mid_prices = get_market_data_range('XJSE', start_date = '2023-01-03', end_date = '2025-12-31',
                                       table_name = 'l1', ticker = 'stock', df_engine = 'polars',
                                      columns = ['Ticker', 'TradeDate', 'LocalTimestamp', 'BidPrice1', 'BidQuantity1', 'AskPrice1', 'AskQuantity1',
                                                 'ExchangeSequenceNo', 'MarketState'])
stock_mid_prices = stock_mid_prices.to_pandas()#toPandas()
stock_mid_prices = stock_mid_prices[stock_mid_prices['MarketState'] == 'CONTINUOUS_TRADING']
stock_mid_prices = stock_mid_prices.sort_values(['LocalTimestamp', 'ExchangeSequenceNo'])
stock_mid_prices['Mid-price'] = (stock_mid_prices['BidPrice1'] + stock_mid_prices['AskPrice1']) / 2
stock_mid_prices.head()

# I can do 3 entire years worth of l1 data at a time for GRT. Takes 60 to load
# I can do 3 entire years worth of l1 data at a time for GFI. Takes 2.5 minutes to load


In [ ]:
N                   = 20
trader_distribution = 'power'
alpha               = 2
identifier          = f'{trader_distribution}_{N}'

In [ ]:
%%time
impact_data = []
for date, day_D in stock.groupby('Date', sort = True):
        print(date)

        trades = day_D.loc[day_D['Price'] != 0]
    
        N = N
        f = af.trader_participation(N = N, method = trader_distribution, alpha = alpha, f_min = 1, f_max = trades.shape[0], seed = 1)
        c = af.cumulative_probs(f)

        if trades.empty:
            continue

        output = af.orders(N = N, trades = trades, cumulative_probs = c)
        for n in range(N):
            
            trader_n_trades = trades.iloc[output[n], ]
           
            if trader_n_trades.empty:
                continue

            trader_n_metaorders = af.metaorders(trader_n_trades)
            
            if len(trader_n_metaorders) < 10:
                continue

            trader_n_features = impact_df_decay(trader_n_metaorders, timing_method = 'immediate')
            trader_n_features['20 AD volatility'] = GRT_AD[GRT_AD['Date'] == date]['20 AD volatility'].iloc[0] 
            trader_n_features['20 AD volume'] = GRT_AD[GRT_AD['Date'] == date]['20 AD volume'].iloc[0]
            
            if not trader_n_features.empty and not trader_n_features.isna().all().all():
                impact_data.append(trader_n_features)



impact_data   = pd.concat(impact_data, ignore_index = True)
exclude_cols = ['RIC', 'Date', 'Start time', 'End time']
numeric_cols = [col for col in impact_data.columns if col not in exclude_cols]
impact_data[numeric_cols] = impact_data[numeric_cols].apply(pd.to_numeric, errors = 'coerce')

impact_data['Date']          = pd.to_datetime(impact_data['Date']).dt.normalize()
impact_data['Start time']    = pd.to_datetime(impact_data['Start time'], format = 'mixed')
impact_data['End time']      = pd.to_datetime(impact_data['End time'], format = 'mixed')
impact_data['duration(min)'] = (impact_data['End time'] - impact_data['Start time']).dt.total_seconds() / 60
durations_GRT               = impact_data['duration(min)']

impact_data.to_csv(f'{ticker}_{identifier}_decay.csv', index = False)
b2.put_file(f'{ticker}_{identifier}_decay.csv', 'test_data')

# takes about 6 minutes to run